**NOTE**: This requires the `unstable` branch of [z3](https://github.com/z3prover/z3). 

I am going to add my git repo of z3 to my sys.path:

In [1]:
import sys
sys.path.insert(0, "/Users/aaronmeurer/Documents/z3/build")

In [2]:
from z3 import Bool, Or, Not, BoolSort, Optimize, IntSort, set_param
from clauses import full_clauses, eq, w
set_param(verbose=10)

The `full_clauses` and `eq` for the example problem `conda install llvmmath anaconda` are in the clauses.py file. 

First translate stuff to z3.

In [3]:
%%time
v = {w[i]: i for i in w}
vars = {i: Bool(i) for i in v}

z3_clauses = []
for clause in full_clauses:
    z3_clause = Or(*[vars[w[i]] if i > 0 else Not(vars[w[-i]]) for i in clause])
    z3_clauses.append(z3_clause)

z3_eq = sum(i*IntSort().cast(vars[w[j]]) for i, j in eq)

CPU times: user 16.1 s, sys: 254 ms, total: 16.4 s
Wall time: 19.4 s


Now set up a z3 optimize object

In [4]:
%%time
o = Optimize()
for z3_clause in z3_clauses:
    o.add(z3_clause)
o.minimize(z3_eq)

CPU times: user 237 ms, sys: 7.24 ms, total: 244 ms
Wall time: 281 ms


And solve it. 

In [5]:
%%time
o.check()
m = o.model()

CPU times: user 2.74 s, sys: 563 ms, total: 3.31 s
Wall time: 4.92 s


In [6]:
sorted([i.sexpr() for i in m if m[i].eq(BoolSort().cast(True))])

['(declare-fun anaconda-1.7.0-np17py33_0.tar.bz2 () Bool)',
 '(declare-fun astropy-0.2.4-np17py33_1.tar.bz2 () Bool)',
 '(declare-fun bitarray-0.8.1-py33_0.tar.bz2 () Bool)',
 '(declare-fun curl-7.30.0-1.tar.bz2 () Bool)',
 '(declare-fun cython-0.19.1-py33_0.tar.bz2 () Bool)',
 '(declare-fun dateutil-2.1-py33_2.tar.bz2 () Bool)',
 '(declare-fun distribute-0.6.45-py33_0.tar.bz2 () Bool)',
 '(declare-fun docutils-0.11-py33_0.tar.bz2 () Bool)',
 '(declare-fun dynd-python-0.4.2-np17py33_0.tar.bz2 () Bool)',
 '(declare-fun flask-0.10.1-py33_1.tar.bz2 () Bool)',
 '(declare-fun freetype-2.4.10-1.tar.bz2 () Bool)',
 '(declare-fun greenlet-0.4.1-py33_0.tar.bz2 () Bool)',
 '(declare-fun hdf5-1.8.9-1.tar.bz2 () Bool)',
 '(declare-fun ipython-1.0.0-py33_0.tar.bz2 () Bool)',
 '(declare-fun itsdangerous-0.23-py33_0.tar.bz2 () Bool)',
 '(declare-fun jinja2-2.7.1-py33_0.tar.bz2 () Bool)',
 '(declare-fun jpeg-8d-1.tar.bz2 () Bool)',
 '(declare-fun libdynd-0.4.2-0.tar.bz2 () Bool)',
 '(declare-fun libne

In [9]:
o.statistics()

(:binary-propagations        1708263
 :conflicts                  2068
 :decisions                  303222
 :dyn-subsumption-resolution 562
 :eliminated-vars            20
 :maxres-cores               75
 :minimized-lits             203
 :mk-binary-clause           34719
 :mk-bool-var                2984
 :mk-clause                  4989
 :mk-ternary-clause          767
 :propagations               1134156
 :ternary-propagations       78762)